## MoE

MoE，全称为Mixed Expert Models，翻译过来就是混合专家模型。MoE 的一个显著优势是能够在远少于 Dense 模型所需的计算资源下进行有效的预训练。这意味着在相同的计算预算条件下，可以显著扩大模型或数据集的规模。特别是在预训练阶段，与 Dense 模型相比，混合专家模型通常能够更快地达到相同的质量水平。在混合专家语言模型中，大部分组件都与传统的 transformers 相同。

MoE 基于 Transformer 架构，主要由两部分组成：
- MoE 层：这些层代替了传统 Transformer 模型中的前馈网络 (FFN) 层。MoE 层包含若干“专家”，每个专家本身是一个独立的神经网络。在实际应用中，这些专家通常是前馈网络 (FFN)，但它们也可以是更复杂的网络结构。
- 门控网络或路由: 这个部分用于决定哪些 token 被发送到哪个专家。例如，在下图中，“More”这个 token 可能被发送到第二个专家，而“Parameters”这个 token 被发送到第一个专家。有时，一个 token 甚至可以被发送到多个专家。token 的路由方式是 MoE 使用中的一个关键点，因为路由器由学习的参数组成，并且与网络的其他部分一同进行预训练。

<div style="text-align:center">
    <img src="Images/MoE_architecture.jpg" alt="MoE architecture" width="500"/>
</div>

总结来说，在混合专家模型 (MoE) 中，将传统 Transformer 模型中的每个前馈网络 (FFN) 层替换为 MoE 层，其中 MoE 层由两个核心部分组成: 一个路由器（或者叫门控网络）和若干数量的专家。

MoE的优点如下：
- 训练速度更快，效果更好。
- 相同参数，推理成本低。
- 扩展性好，允许模型在保持计算成本不变的情况下增加参数数量，这使得它能够扩展到非常大的模型规模，如万亿参数模型。
- 多任务学习能力，MoE 在多任务学习中具备很好的新能。

MoE的缺点如下：
- 训练稳定性，MoE在训练过程中可能会遇到稳定性问题。
- 通信成本，在分布式训练环境中，MoE 的专家路由机制可能会增加通信成本，尤其是在模型规模较大时。
- 模型复杂性，MoE 的设计相对复杂，可能需要更多的工程努力来实现和优化。
- 下游任务性能，MoE由于其稀疏性，使得在 Fine-tuning 过程中容易出现过拟合。

主要介绍几个模块的实现：
- self-attention 以及 multi-head self-attention 模块的实现
- 稀疏混合专家代替单独的前馈神经网络
- Top-k 门控和有噪声的 Top-k 门控

实现代码参考 [makeMoE](https://github.com/AviSoori1x/makeMoE) 项目。

## self-attention

常规的 self-attention 实现方式使用的缩放点积自注意力，查询矩阵、键矩阵和值矩阵都来自相同的输入序列，同时为了确保自回归语言生成过程的完整性，特别是在纯解码器模型中，使用了一种因果自注意力，也叫因果掩码。它可以掩盖当前 token 所处位置之后的任何信息，从而引导模型只关注序列的前面部分。值得注意的是，稀疏混合专家模型并不局限于仅有解码器的 Transformer 架构。事实上，这一领域的许多重要的成果都是围绕 T5 架构展开的，T5 架构也包含了 Transformer 模型中的编码器和解码器组件。

In [21]:
# 首先先实现 self-attention 模块的工作原理代码
# 创建一个 [batch_size, seq_len, hidden_dim] 的张量，命名为 x
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(42)
batch_size, seq_len, n_embed = 4, 8, 32
x = torch.randn(batch_size, seq_len, n_embed)

# 接下来实现一个单头的 self-attention 模块的工作原理代码
head_size = 16 # 每个头的维度
key = nn.Linear(n_embed, head_size, bias=False)
query = nn.Linear(n_embed, head_size, bias=False)
value = nn.Linear(n_embed, head_size, bias=False)
k = key(x)   # (4, 8, 16)
q = query(x) # (4, 8, 16)
weight = q @ k.transpose(-2, -1) # (4, 8, 16) @ (4, 16, 8) ---> (4, 8, 8)

# 为了保证每个 token 只能关注到其自身以及前面几个 token，需要对 weight 进行 mask，即使用一个 [seq_len, seq_len] 的因果掩码
tril = torch.tril(torch.ones(seq_len, seq_len))
print("tril: \n{}".format(tril))
weight = weight.masked_fill(tril == 0, float('-inf'))
weight = F.softmax(weight, dim=-1) # (4, 8, 8)

v = value(x) # (4, 8, 16)
out = tril @ v # (4, 8, 8) @ (4, 8, 16) -> (4, 8, 16)

tril: 
tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])


In [22]:
# 接来下将相关模块整合成一个函数，方便调用

# 单头的自注意力机制
class OneHead(nn.Module):
    """ one head of self-attention """

    def __init__(self, n_embed, head_size, seq_len, dropout=0.1):
        super().__init__()
        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.value = nn.Linear(n_embed, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(seq_len, seq_len)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        batch_size, seq_len, n_embed = x.shape
        k = self.key(x)   # (batch_size, seq_len, head_size)
        q = self.query(x) # (batch_size, seq_len, head_size)
        # compute attention scores ("affinities")
        weight = q @ k.transpose(-2,-1) * n_embed ** -0.5 # (batch_size, seq_len, head_size) @ (batch_size, head_size, seq_len) -> (batch_size, seq_len, seq_len)
        weight = weight.masked_fill(self.tril[:seq_len, :seq_len] == 0, float('-inf')) # (batch_size, seq_len, seq_len)
        weight = F.softmax(weight, dim=-1) # (batch_size, seq_len, seq_len)
        weight = self.dropout(weight)
        # perform the weighted aggregation of the values
        v = self.value(x) # (batch_size, seq_len, head_size)
        out = weight @ v # (batch_size, seq_len, seq_len) @ (batch_size, seq_len, head_size) -> (batch_size, seq_len, head_size)
        return out

# 基于单头的自注意力机制实现多头的自注意力机制
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, n_embed, num_heads, head_size, dropout):
        super().__init__()
        self.heads = nn.ModuleList([OneHead(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embed, n_embed)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

## 专家模块 - 多层感知器

在稀疏混合专家架构中，每个 Transformer 区块内的自注意力机制保持不变。不过，每个区块的结构发生了巨大的变化：标准的前馈神经网络被多个稀疏激活的前馈网络（即专家网络）所取代。所谓「稀疏激活」，是指序列中的每个 token 只被分配给有限数量的专家（通常是一个或两个）。

这有助于提高训练和推理速度，因为每次前向传递都会激活少数专家。不过，所有专家都必须存在 GPU 内存中，因此当参数总数达到数千亿甚至数万亿时，就会产生部署方面的问题。

In [34]:
# 专家模块的实现和FFN模块的实现相同
class Expert(nn.Module):
    """ An MLP is a simple linear layer followed by a non-linearity i.e. each Expert """

    def __init__(self, n_embed, dropout):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed, 4 * n_embed),
            nn.ReLU(),
            nn.Linear(4 * n_embed, n_embed),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

## Top-k 门控

门控网络，也称为路由，确定哪个专家网络接收来自多头注意力的 token 的输出。假设有 4 个专家，token 需要被路由到前 2 个专家中。首先需要通过线性层将 token 输入到门控网络中。该层将对应于（batch_size，seq_len，n_embed）的输入张量从（4，8，32）维度，投影到对应于（batch_size，seq_len，num_expert）的新形状：（4、8，4）。其中 n_embed 是输入的通道维度，num_experts 是专家网络的计数。然后，沿最后一个维度，找出最大的前两个值及其相应的索引。

<div style="text-align:center">
    <img src="Images/top_k_gating.jpg" alt="top-k gating" width="800"/>
</div>

In [27]:
# 通过一个简单的例子来理解 top-k 门控机制
num_experts = 4
top_k = 2
batch_size = 4
seq_len = 8
n_embed = 32

# 假如经过 multi-head self attention 之后，得到一个 (4, 8, 32) 的张量
mh_output = torch.randn(batch_size, seq_len, n_embed)

topkgate_linear = nn.Linear(n_embed, num_experts) # nn.Linear(32, 4)

logits = topkgate_linear(mh_output)
top_k_logits, top_k_indices = logits.topk(top_k, dim=-1)  # Get top-k experts
top_k_logits, top_k_indices

(tensor([[[ 1.0700,  0.7206],
          [ 0.2494, -0.0838],
          [ 1.2022,  0.5802],
          [ 0.8623,  0.6392],
          [ 0.3154,  0.0610],
          [ 0.8664,  0.6319],
          [ 0.5692,  0.0469],
          [ 1.3120, -0.3133]],
 
         [[ 1.2228, -0.0321],
          [ 1.1416,  0.3027],
          [ 0.5253,  0.4374],
          [ 0.1580, -0.0446],
          [ 0.3139,  0.2930],
          [ 0.3529,  0.2312],
          [ 1.4150,  0.2912],
          [ 0.5945,  0.1327]],
 
         [[ 0.5750, -0.0629],
          [ 0.6928,  0.2333],
          [ 0.6365,  0.2649],
          [ 0.4032,  0.1236],
          [ 0.8245, -0.1826],
          [ 1.3292,  0.2458],
          [-0.0589, -0.0794],
          [ 0.8956,  0.6806]],
 
         [[ 0.1375,  0.0218],
          [ 0.4306,  0.0555],
          [ 1.3460,  0.9864],
          [ 0.9852,  0.1215],
          [ 1.0872,  0.1047],
          [ 0.1002, -0.1142],
          [ 1.3141, -0.5620],
          [ 0.9606, -0.1521]]], grad_fn=<TopkBackward0>),
 te

通过仅保留沿最后一个维度进行比较的前 k 大的值，来获得稀疏门控的输出。用负无穷值填充其余部分，在使用 softmax 激活函数。负无穷会被映射至零，而最大的前两个值会更加突出，且和为 1。要求和为 1 是为了对专家输出的内容进行加权。

In [25]:
zeros = torch.full_like(logits, float('-inf')) 
sparse_logits = zeros.scatter(-1, top_k_indices, top_k_logits)
sparse_logits

tensor([[[   -inf,  0.2479,    -inf,  0.9578],
         [-0.4220, -0.4273,    -inf,    -inf],
         [ 0.1390,    -inf,    -inf,  0.1366],
         [   -inf,  0.2133,    -inf,  0.2477],
         [   -inf,  0.4563,  0.7880,    -inf],
         [ 0.1496,    -inf,    -inf, -0.0354],
         [-0.0292,    -inf,  1.0569,    -inf],
         [ 0.5379,    -inf,  0.1495,    -inf]],

        [[ 0.5540,    -inf,  0.2338,    -inf],
         [ 0.2380,    -inf,  0.8278,    -inf],
         [   -inf,  0.1439,  0.6391,    -inf],
         [ 0.2268,  0.4964,    -inf,    -inf],
         [ 0.1168, -0.1425,    -inf,    -inf],
         [ 0.3889,  0.4419,    -inf,    -inf],
         [ 1.0024,    -inf,  0.0241,    -inf],
         [   -inf, -0.1975,  0.5915,    -inf]],

        [[ 0.6596,    -inf,    -inf,  0.7077],
         [   -inf,    -inf,  0.6383,  0.7035],
         [   -inf,    -inf,  0.2839,  1.1395],
         [ 0.2079,    -inf,  0.5646,    -inf],
         [   -inf,  0.0644,  0.8486,    -inf],
         

In [26]:
gating_output= F.softmax(sparse_logits, dim=-1)
gating_output

tensor([[[0.0000, 0.3296, 0.0000, 0.6704],
         [0.5013, 0.4987, 0.0000, 0.0000],
         [0.5006, 0.0000, 0.0000, 0.4994],
         [0.0000, 0.4914, 0.0000, 0.5086],
         [0.0000, 0.4178, 0.5822, 0.0000],
         [0.5461, 0.0000, 0.0000, 0.4539],
         [0.2523, 0.0000, 0.7477, 0.0000],
         [0.5959, 0.0000, 0.4041, 0.0000]],

        [[0.5794, 0.0000, 0.4206, 0.0000],
         [0.3567, 0.0000, 0.6433, 0.0000],
         [0.0000, 0.3787, 0.6213, 0.0000],
         [0.4330, 0.5670, 0.0000, 0.0000],
         [0.5645, 0.4355, 0.0000, 0.0000],
         [0.4868, 0.5132, 0.0000, 0.0000],
         [0.7268, 0.0000, 0.2732, 0.0000],
         [0.0000, 0.3124, 0.6876, 0.0000]],

        [[0.4880, 0.0000, 0.0000, 0.5120],
         [0.0000, 0.0000, 0.4837, 0.5163],
         [0.0000, 0.0000, 0.2983, 0.7017],
         [0.4118, 0.0000, 0.5882, 0.0000],
         [0.0000, 0.3134, 0.6866, 0.0000],
         [0.5710, 0.0000, 0.0000, 0.4290],
         [0.0000, 0.0000, 0.6663, 0.3337],
       

In [28]:
# 将 top-k 门控机制整理成一个函数
class TopkRouter(nn.Module):
    def __init__(self, n_embed, num_experts, top_k):
        super(TopkRouter, self).__init__()
        self.top_k = top_k
        self.linear =nn.Linear(n_embed, num_experts)
    
    def forward(self, mh_output):
        # mh_ouput is the output tensor from multi-head self attention block
        logits = self.linear(mh_output)
        top_k_logits, indices = logits.topk(self.top_k, dim=-1)
        zeros = torch.full_like(logits, float('-inf'))
        sparse_logits = zeros.scatter(-1, indices, top_k_logits)
        router_output = F.softmax(sparse_logits, dim=-1)
        return router_output, indices

In [30]:
# 测试用例
num_experts = 4
top_k = 2
batch_size = 4
seq_len = 8
n_embed = 32

mh_output = torch.randn(batch_size, seq_len, n_embed)
top_k_gate = TopkRouter(n_embed, num_experts, top_k)
gating_output, indices = top_k_gate(mh_output)
gating_output.shape, gating_output, indices

(torch.Size([4, 8, 4]),
 tensor([[[0.4262, 0.0000, 0.0000, 0.5738],
          [0.0000, 0.3942, 0.0000, 0.6058],
          [0.0000, 0.5768, 0.4232, 0.0000],
          [0.3226, 0.0000, 0.0000, 0.6774],
          [0.3806, 0.0000, 0.6194, 0.0000],
          [0.7507, 0.0000, 0.2493, 0.0000],
          [0.0000, 0.0000, 0.4478, 0.5522],
          [0.0000, 0.0000, 0.3864, 0.6136]],
 
         [[0.2937, 0.7063, 0.0000, 0.0000],
          [0.3889, 0.6111, 0.0000, 0.0000],
          [0.0000, 0.5416, 0.4584, 0.0000],
          [0.0000, 0.0000, 0.4122, 0.5878],
          [0.3586, 0.0000, 0.6414, 0.0000],
          [0.2137, 0.0000, 0.0000, 0.7863],
          [0.3996, 0.0000, 0.6004, 0.0000],
          [0.0000, 0.6880, 0.0000, 0.3120]],
 
         [[0.5106, 0.4894, 0.0000, 0.0000],
          [0.0000, 0.5588, 0.4412, 0.0000],
          [0.0000, 0.3291, 0.6709, 0.0000],
          [0.0000, 0.5787, 0.0000, 0.4213],
          [0.5344, 0.0000, 0.0000, 0.4656],
          [0.3891, 0.0000, 0.6109, 0.0000],
  

## 有噪声的 Top-k 门控 —— 实现负载平衡

有噪声的 Top-k 门控机制是训练 MoE 模型的一个重要工具。从本质上讲，不会希望所有的 token 都发送给同一组「受欢迎」的专家网络。人们需要的是能在开发和探索之间取得良好平衡。为此，为了负载平衡，从门控的线性层向 logits 激活函数添加标准正态噪声是有帮助的，这使训练更有效率。

<div style="text-align:center">
    <img src="Images/noised_top_k_gating.jpg" alt="noised top-k gating" width="800"/>
</div>

In [29]:
class NoisyTopkRouter(nn.Module):
    def __init__(self, n_embed, num_experts, top_k):
        super(NoisyTopkRouter, self).__init__()
        self.top_k = top_k
        # layer for router logits
        self.topkroute_linear = nn.Linear(n_embed, num_experts)
        self.noise_linear =nn.Linear(n_embed, num_experts)

    def forward(self, mh_output):
        # mh_output is the output tensor from multihead self attention block
        logits = self.topkroute_linear(mh_output)

        # Noise logits
        noise_logits = self.noise_linear(mh_output)

        # Adding scaled unit gaussian noise to the logits
        noise = torch.randn_like(logits)*F.softplus(noise_logits)
        noisy_logits = logits + noise

        top_k_logits, indices = noisy_logits.topk(self.top_k, dim=-1)
        zeros = torch.full_like(noisy_logits, float('-inf'))
        sparse_logits = zeros.scatter(-1, indices, top_k_logits)
        router_output = F.softmax(sparse_logits, dim=-1)
        return router_output, indices

In [31]:
# 测试用例
num_experts = 8
top_k = 2
batch_size = 4
seq_len = 8
n_embed = 32

mh_output = torch.randn(batch_size, seq_len, n_embed)
noisy_top_k_gate = NoisyTopkRouter(n_embed, num_experts, top_k)
gating_output, indices = noisy_top_k_gate(mh_output)
gating_output.shape, gating_output, indices

(torch.Size([4, 8, 8]),
 tensor([[[0.0000, 0.5102, 0.0000, 0.0000, 0.0000, 0.4898, 0.0000, 0.0000],
          [0.4597, 0.0000, 0.0000, 0.5403, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.5533, 0.0000, 0.4467, 0.0000, 0.0000],
          [0.0000, 0.5424, 0.0000, 0.4576, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0710, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.9290],
          [0.0000, 0.0000, 0.9008, 0.0000, 0.0000, 0.0992, 0.0000, 0.0000],
          [0.7085, 0.0000, 0.0000, 0.0000, 0.2915, 0.0000, 0.0000, 0.0000],
          [0.3907, 0.0000, 0.6093, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],
 
         [[0.0000, 0.0000, 0.3858, 0.0000, 0.0000, 0.6142, 0.0000, 0.0000],
          [0.5314, 0.0000, 0.0000, 0.4686, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2911, 0.0000, 0.7089],
          [0.0000, 0.0000, 0.0000, 0.6592, 0.0000, 0.0000, 0.3408, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.4964, 0.0000, 0.0000, 0.

## 稀疏化的混合专家模块

在获得门控网络的输出结果之后，对于给定的 token，将前 k 个值选择性地与来自相应的前 k 个专家的输出相乘。这种选择性乘法的结果是一个加权和，该加权和构成 SparseMoe 模块的输出。这个过程的关键和难点是避免不必要的乘法运算，只为前 k 名专家进行正向转播。为每个专家执行前向传播将破坏使用稀疏 MoE 的目的，因为这个过程将不再是稀疏的。

In [35]:
class SparseMoE(nn.Module):
    def __init__(self, n_embed, num_experts, top_k, dropout):
        super(SparseMoE, self).__init__()
        self.router = NoisyTopkRouter(n_embed, num_experts, top_k)
        self.experts = nn.ModuleList([Expert(n_embed, dropout) for _ in range(num_experts)])
        self.top_k = top_k

    def forward(self, x):
        gating_output, indices = self.router(x)
        final_output = torch.zeros_like(x)

        # Reshape inputs for batch processing
        flat_x = x.view(-1, x.size(-1))
        flat_gating_output = gating_output.view(-1, gating_output.size(-1))

        # Process each expert in parallel
        for i, expert in enumerate(self.experts):
            # Create a mask for the inputs where the current expert is in top-k
            expert_mask = (indices == i).any(dim=-1)
            flat_mask = expert_mask.view(-1)

            if flat_mask.any():
                expert_input = flat_x[flat_mask]
                expert_output = expert(expert_input)

                # Extract and apply gating scores
                gating_scores = flat_gating_output[flat_mask, i].unsqueeze(1)
                weighted_output = expert_output * gating_scores

                # Update final output additively by indexing and adding
                final_output[expert_mask] += weighted_output.squeeze(1)

        return final_output

In [36]:
# 测试用例
num_experts = 8
top_k = 2
batch_size = 4
seq_len = 8
n_embed = 32
dropout = 0.1

mh_output = torch.randn(batch_size, seq_len, n_embed)
sparse_moe = SparseMoE(n_embed, num_experts, top_k, dropout)
final_output = sparse_moe(mh_output)
print("Shape of the final output:", final_output.shape)

Shape of the final output: torch.Size([4, 8, 32])


## 模块整合

将多头自注意力和稀疏混合专家相结合，形成稀疏混合专家 Transformer 块。就像在 vanilla transformer 块中一样，也要使用残差以确保训练稳定，并避免梯度消失等问题。此外，要采用层归一化来进一步稳定学习过程。

In [37]:
class Block(nn.Module):
    """ Mixture of Experts Transformer block: communication followed by computation (multi-head self attention + SparseMoE) """

    def __init__(self, n_embed, n_head, num_experts, top_k, dropout):
        # n_embed: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embed // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.smoe = SparseMoE(n_embed, num_experts, top_k, dropout)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.smoe(self.ln2(x))
        return x

最后，将所有内容整合在一起，形成稀疏混合专家语言模型。

In [38]:
class SparseMoELanguageModel(nn.Module):
    def __init__(self, vocab_size, n_embed, block_size, n_head, n_layer, num_experts, top_k):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.vocab_size = vocab_size
        self.n_embed = n_embed
        self.block_size = block_size
        self.n_head = n_head
        self.n_layer = n_layer
        self.num_experts = num_experts
        self.top_k = top_k
        
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table = nn.Embedding(block_size, n_embed)
        self.blocks = nn.Sequential(*[Block(n_embed, n_head=n_head, num_experts=num_experts, top_k=top_k) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embed) # final layer norm
        self.lm_head = nn.Linear(n_embed, vocab_size)

    def forward(self, idx, targets=None):
        batch_size, seq_len = idx.shape

        # idx and targets are both (batch_size, seq_len) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (batch_size, seq_len, n_embed)
        pos_emb = self.position_embedding_table(torch.arange(seq_len)) # (seq_len, n_embed)
        x = tok_emb + pos_emb # (batch_size, seq_len, n_embed)
        x = self.blocks(x) # (batch_size, seq_len, n_embed)
        x = self.ln_f(x) # (batch_size, seq_len, n_embed)
        logits = self.lm_head(x) # (batch_size, seq_len, vocab_size)

        if targets is None:
            loss = None
        else:
            batch_size, seq_len, n_embed = logits.shape
            logits = logits.view(batch_size*seq_len, n_embed)
            targets = targets.view(batch_size*seq_len)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -self.block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx